<a href="https://colab.research.google.com/github/jugernaut/MACTI-programacionparalelo/blob/main/03_CUDA/02_Algoritmos_CUDA_SCP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<font color="Teal" face="Comic Sans MS,arial">
  <h1 align="center"><i>Algoritmos CUDA</i></h1>
  </font>
  <font color="Black" face="Comic Sans MS,arial">
  <h5 align="center"><i>Profesor: M. en C. Miguel Angel Pérez León</i></h5>
  <h5 align="center"><i>Ayudante: Lucía Martínez Rivas</i></h5>
  <h5 align="center"><i>Ayudante: Erick Jesús Rios Gonzalez</i></h5>
  <h5 align="center"><i>Materia: Seminario de programación en paralelo</i></h5>
  </font>

# Introducción

Como ya se menciono previamente, *CUDA* hace uso de las *GPU's* (dispositivos de cómputo de propósito especifico), estos dispositivos están optimizados para trabajar con imágenes y resulta que una imagen dentro de una computadora se representa mediante elementos matemáticos como matrices o vectores.

De hecho la mayoría de los formatos de imágenes más comunes (*.jpg, .jpeg, .png*) consideran a la imagen como una matriz de pixeles o un mapa de bits.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Imagenes/CUDA/smile.png?raw=1" width="600">
</center>



# Suma de Vectores

El algoritmo más sencillo que podemos comenzar a analizar es la suma de vectores y aunque suene trivial, la realidad es que hasta antes de este momento toda suma de vectores que hayas realizado previamente **se ejecuto de manera secuencial**, lo cuál significa un desperdicio de recursos.

Gracias a *CUDA* (aunque también se puede realizar con *OpenMP* y *MPI*) esta operación elemental se puede realizar en paralelo y gracias a esto optimizar recursos, lo que se traduce en un menor tiempo de ejecución.

## ¿Cómo funciona?

La forma tradicional de como funciona la suma de vectores, se basa en la definición formal "se realiza entrada a entrada", sin embargo nunca nos dijeron que esta "suma entrada a entrada" se puede realizar en paralelo.

<center>
<img src="https://github.com/jugernaut/ProgramacionEnParalelo/blob/desarrollo/Imagenes/CUDA/sumavect.png?raw=1" width="600">
</center>


In [1]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-sfe9f18c
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-sfe9f18c
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 801584cceb559adc54e828ebe9b385c5f53fe70f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for nvcc4jupyter: filename=nvcc4jupyter-1.2.1-py3-none-any.whl size=10743 sha256=3a570d3ab74e9051a615aa78d54722c78a69a0c83a1d407fccde14c37a30e96c
  Stored in directory: /tmp/pip-ephem-wheel-cache-ziztyl4x/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built nvcc4jupyter


In [4]:
%load_ext nvcc4jupyter

Detected platform "Colab". Running its setup...
Source files will be saved in "/tmp/tmpxdxikldg".


In [6]:
%%cuda

#include "cuda_runtime.h"
#include "device_launch_parameters.h"
#include <stdio.h>

// Kernel (funcion) que se invoca desde el Host y se ejecuta en un dispositivo
__global__ void suma_vectores(int* c, const int* a, const int* b, int size) {
    // polinomio de direccionamiento
    // ¡¡OJO 2 bloques(0 y 1), dim = 3, thread por bloque de 0-2!!
    int i = blockDim.x * blockIdx.x + threadIdx.x;
    if (i < size) {
        // descomnetar para debugear
        //printf("%d \n",blockIdx.x);
        //printf("%d \n",blockDim.x);
        //printf("%d \n",threadIdx.x);
        c[i] = a[i] + b[i];
    }
}

// Funcion auxiliar que encapsula la suma con CUDA
void suma_CUDA(int* c, const int* a, const int* b, int tam) {
    int* dev_a = nullptr;
    int* dev_b = nullptr;
    int* dev_c = nullptr;

    // Reservamos espacio de memoria para los datos, 2 de entrada y una salida
    cudaMalloc((void**)&dev_c, tam * sizeof(int));
    cudaMalloc((void**)&dev_a, tam * sizeof(int));
    cudaMalloc((void**)&dev_b, tam * sizeof(int));

    // Copiamos los datos de entrada desde el CPU a la memoria del GPU
    cudaMemcpy(dev_a, a, tam * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_b, b, tam * sizeof(int), cudaMemcpyHostToDevice);

    // Se invoca al kernel en el GPU con un hilo por cada elemento
    // 2 es el numero de bloques y (tam + 1)/2 es el numero de hilos en cada bloque
    suma_vectores<<<2, (tam + 1) / 2>>>(dev_c, dev_a, dev_b, tam);

    // Esta funcion espera a que termine de ejecutarse el kernel y
    // devuelve los errores que se hayan generado al ser invocado
    cudaDeviceSynchronize();

    // Copiamos el vector resultado de la memoria del GPU al CPU
    cudaMemcpy(c, dev_c, tam * sizeof(int), cudaMemcpyDeviceToHost);

    // Se libera la memoria empleada
    cudaFree(dev_c);
    cudaFree(dev_a);
    cudaFree(dev_b);
}

// Funcion principal que sirve de prueba para el algoritmo
int main(int argc, char** argv) {

    // Datos de entrada para nuestra funcion
    const int tam = 5;
    const int a[tam] = {  1,  2,  3,  4,  5 };
    const int b[tam] = { 10, 20, 30, 40, 50 };
    int c[tam] = { 0 };

    // Se llama a la funcion que encapsula el Kernel
    suma_CUDA(c, a, b, tam);

    // Mostramos resultado
    printf("{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {%d, %d, %d, %d, %d}\n", c[0], c[1], c[2], c[3], c[4]);

    // Se liberan recursos
    cudaDeviceReset();

    return 0;
}

{1, 2, 3, 4, 5} + {10, 20, 30, 40, 50} = {11, 22, 33, 44, 55}



## Ventajas

Debido a lo que ya conocemos respecto al funcinamiento y desempeño de *CUDA*, podemos afirmar que este tipo de operaciones (suma de vectores) se realizan de manera más sencilla en un *GPU*.

Sean $\vec{a}=\{1,2,3,4,5\}$ y $\vec{b}=\{10,20,30,40,50\}$ entonces $\left(a_{0},b_{0}\right)$ se envían al núcleo 0, $\left(a_{1},b_{1}\right)$ se envían al núcleo 1, así sucesivamente hasta $\left(a_{n-1},b_{n-1}\right)$ se envían al núcleo $n-1$. En caso de que existan más entradas que nucleos, entonces se tendría que esperar a liberar alguno de los núcleos previamente empleados, sin embargo debido a la arquitectura de los *GPU's* sabemos que cada *GPU* posee muchos nucleos, **en las tarjetas más recientes podemos hablar del orden de miles**.

De igual manera como se midió el tiempo de ejecución con *OpenMP* o *MPI*, existe forma de medir el tiempo de ejecución de los algoritmos usando *CUDA*.

¿Cuáles son los ordenedes de complejidad a los que pertenecen ambas versiones de la suma de vectores, secuencial y en paralelo?.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/Cuda/gpucpu.jpg?raw=1" width="600">
</center>


## Desventajas

La desventaja más notoria en el algoritmo anterior, es el **cuello de botella** que se genera tanto al enviar los datos a procesar al *GPU*, como al extraerlos del *GPU*, sin embargo es claro que a mayor cantidad de datos a procesar también se tendría una mayor ganancia en tiempo, lo que se traduce en un menor tiempo de ejecución.

Este tipo de características (gran cantidad de datos a procesar) normalmente se encuentran en muchas áreas de las ciencias e ingenierías, por ejemplo ***machine learning***, por mencionar alguna.

# Operaciones con matrices

Una vez que ya se comprendió el desempeño de *CUDA* para la suma de vectores, es sencillo extender su aplicación a operaciones con matrices, por ejemplo la **suma de matrices**.

Por lo general cuando se hace uso de modelos matemáticos, estos toman la forma de matriz, uno de los más conocidos son las **redes neuronales**. Estas redes neuronales toman forma de matriz y en cada entrada de la matriz se tiene una neurona, y a su vez cada neurona puede ser representada por un escalar o un vector o incluso otra matriz.

Es por este motivo que las *GPU's* y en particular *CUDA* ha mostrado un excelente desempeño en el proceso de entrenamiento de las redes neuronales, más recientemente han surgido nuevos dispositivos de cómputo especifico como los *TPU's* (unidades de procesamiento tensorial), dispositivos que desde que fueron diseñados se contemplo su uso para el área de inteligencia artificial.

Volviendo al funcionamiento de *CUDA*, veamos una parte fundamental del mismo, la **jerarquía de memoria** de *CUDA* se estructura de la siguiente manera.

<center>
<img src="https://github.com/jugernaut/Numerico2021/blob/master/Imagenes/Cuda/memtotal.png?raw=1" width="600">
</center>

## Entendiendo CUDA

Queda como ejercicio, realizar la suma de matrices empleando *CUDA*.



## Hint (polinomio de direccionamiento)

Imaginemos que queremos "aplanar" una matríz para representarla con un vector, es decir que necesitamos almacenar (y recuperar) cada una de las entradas de una matriz en un vector, ¿cómo le hacemos?.

La respuesta es mediante el **polinomio de direccionamiento**, este polinomio nos indica mediante una sola entrada, la localidad del vector que le corresponde a cada uno de los elementos de una matriz.

Supongamos que queremos almacenar los elementos de la matriz $A$ en una lista o vector.

Sea

$$A\in M_{2x2}=\left(\begin{array}{cc}
3_{(0,0)} & 6_{(0,1)}\\
7_{(1,0)} & 9_{(1,1)}
\end{array}\right)$$

La idea sería que estos elementos se alamcenen de la siguiente forma.

$$A=\left[\begin{array}{cccc}
3 & 6 & 7 & 9\end{array}\right]$$

Nos gustaría que la entrada $(0,0)$ de $A$ fuera mapeada a la localidad 0 de la lista y así sucesivamente hasta llegar a que la entrada $(1,1)$ se mapeara a la localidad 3 del arreglo, es decir

\begin{array}{cc}
f((0,0))=0 & f((0,1))=1\\
f((1,0))=2 & f((1,1))=3
\end{array}

Podríamos pensar que una buena forma de definir a $f$, seria $f((x,y))=x+y$, pero veamos que sucede al probarla.

\begin{array}{c}
f((0,0))=0+0=0.......\text{¡bien!}\\
f((0,1))=0+1=1.......\text{¡bien!}
\end{array}

Vamos bien, veamos que sucede con los elementos restantes.

\begin{array}{c}
f((1,0))=1+0=1.......\text{¡colisión!}\\
f((0,1))=1=f((1,0))
\end{array}

Dado que se tuvo una colisión, es necesario re-definirla de otra manera menos ingenua. Veamos que sucede si definimos a $f$ de la siguiente manera.

$$f((x,y))=2x+y$$

Al probarla, lo que obtenemos es.

\begin{array}{c}
f((0,0))=2*0+0=0\\
f((0,1))=2*0+1=1\\
f((1,0))=2*1+0=2\\
f((1,1))=2*1+1=3
\end{array}

Esta función no muestra colisiones (al menos en el dominio y codominio definidos), incluso se podría probar que no presentará colisiones para ningún par de tuplas de naturales.

Así que podemos pensar, que para el caso particular de matrices bidimensionales $A_{(i,j)}\in M_{ren \times col}$ podemos definir la función hash (polinomio de direccionamiento) que mapea localidades de dicha matriz en una lista (arreglo) unidimensional de la siguiente forma.

$$f((i,j))=col*i+j$$

¿Podemos extender este polinomio a objetos de 3 dimensiones, largo, ancho, profundidad?.

# Glosario

*Host*: En el contexto de *CUDA* el host es el *CPU* del dispositivo de cómputo en el que se ejecuta el algoritmo.

Asíncrono: En computación un evento (proceso) asíncrono es aquel no tiene correspondencia temporal con otro evento.

# Referencias

1. Tolga Soyata: GPU Parallel Program Development Using Cuda.
2. https://fisica.cab.cnea.gov.ar/gpgpu/images/clases/clase_1_cuda.pdf
3. Dongarra Foster: Source Book of parallel computing.